In [1]:
import pandas as pd
import json
from time import sleep
from pymongo import MongoClient, errors

from icecream import ic

In [2]:
host = 'localhost'
port = 27017
db_name = 'oquant_runtime'
coll_name = '1234:group_prices'
strategy_id = '1'

client = MongoClient(host, port, serverSelectionTimeoutMS=5000)
db = client[db_name]
coll = db[coll_name]

all_groups = coll.find({'strategy_id': strategy_id})



In [3]:
def to_straddle_group(prices):
    def get_key_from_straddle_prices(price_data):
        if len(price_data) == 3:
            symbols = list(price_data.keys())
            (head, tail) = (symbols[1], symbols[2]) if symbols[1] > symbols[2] else (symbols[2], symbols[1])
            return f'{head}-{tail}' 
        return None
        
    def split(prices):
        all_data = {}    
        for dt, data in prices.items():
            if len(data) == 3:
                key = (get_key_from_straddle_prices(data))
                new_data = {'datetime': dt}
                new_data.update(data)
                if key in all_data:
                    all_data[key].append(new_data)
                else:
                    all_data[key] = [new_data]
        return all_data
    
    def to_df(all_data):
        all_df = {}
        for code, data in all_data.items():
            df = pd.DataFrame(data)
            df.set_index('datetime', inplace=True)
            all_df[code] = df
        return all_df
    
    all_data = split(prices)
    all_df = to_df(all_data)
    return all_df

In [4]:
for group in all_groups:
    prices = group['prices']
    prices = json.loads(prices)
    straddle_group = to_straddle_group(prices)
    
    break

In [5]:
profits = []
for code, df in straddle_group.items():
    symbols = code.split('-')
    df['total'] = df[symbols[0]] + df[symbols[1]]
    df['total'] = df['total'].bfill()
    
    profit = 10000 * (df['total'].iloc[0] - df['total'].iloc[-1])
    profit = round(10000 * (df['total'].iloc[0] - df['total'].iloc[-1]), 2)
    profits.append(profit)

In [ ]:
straddle_group

In [7]:
profits

[-126.0, -373.0, -2470.0, 655.0, 351.0]

In [8]:
sum(profits)

-1963.0

In [13]:
for dt, p in prices.items():
    # ic(dt)
    # break
    if dt.startswith('2020-06-24 14'):
        ic(dt, p)

ic| dt: '2020-06-24 14:00:00'
    p: {'10002535.SH': 0.0315, '10002538.SH': 0.0096, '510050.SH': 2.951}
ic| dt: '2020-06-24 14:01:00'
    p: {'10002535.SH': 0.0316, '10002538.SH': 0.0096, '510050.SH': 2.953}
ic| dt: '2020-06-24 14:02:00'
    p: {'10002535.SH': 0.0314, '10002538.SH': 0.0095, '510050.SH': 2.952}
ic| dt: '2020-06-24 14:03:00'
    p: {'10002535.SH': 0.0314, '10002538.SH': 0.0097, '510050.SH': 2.95}
ic| dt: '2020-06-24 14:04:00'
    p: {'10002535.SH': 0.0312, '10002538.SH': 0.0097, '510050.SH': 2.95}
ic| dt: '2020-06-24 14:05:00'
    p: {'10002535.SH': 0.0313, '10002538.SH': nan, '510050.SH': 2.95}
ic| dt: '2020-06-24 14:06:00'
    p: {'10002535.SH': 0.0314, '10002538.SH': 0.0096, '510050.SH': 2.951}
ic| dt: '2020-06-24 14:07:00'
    p: {'10002535.SH': 0.0315, '10002538.SH': 0.0095, '510050.SH': 2.951}
ic| dt: '2020-06-24 14:08:00'
    p: {'10002535.SH': 0.0315, '10002538.SH': nan, '510050.SH': 2.95}
ic| dt: '2020-06-24 14:09:00'
    p: {'10002535.SH': 0.0317, '10002538.SH'